## Part 1

In [ ]:
TEST_INFILE = "inputs/day_5_test_1.txt"
INFILE = "inputs/day_5_1.txt"

with open(TEST_INFILE) as infile:
#with open(INFILE) as infile:
    lines = infile.read().split("\n\n")

In [ ]:
maps = {line.split(":")[0]: line.split(":")[1].strip().split("\n") for line in lines}

In [ ]:
class RangeMap:
    def __init__(self, dest_start, src_start, length):
        self.dest_start = dest_start
        self.source_start = src_start
        self.length = length

    def includes_src(self, num):
        if num >= self.source_start and num < self.source_start + self.length:
            return True
        return False

    def get_dest_from_src(self, src):
        if self.includes_src(src):
            return self.dest_start + (src - self.source_start)
        return None

In [ ]:
seeds = [int(s) for s in maps.pop("seeds")[0].split()]

In [ ]:
maps

In [ ]:
for map_name, ranges in maps.items():
    range_objs = []
    for range in ranges:
        range_objs.append(RangeMap(*[int(r) for r in range.split()]))
    maps[map_name] = range_objs

In [ ]:
def map(from_num, map_layer_name):
    this_map = maps[map_layer_name]
    for this_map_row in this_map:
        mapped = this_map_row.get_dest_from_src(from_num)
        if mapped is not None:
            break

    if not mapped:
        mapped = from_num

    return mapped

def map_seed(seed):
    soil = map(seed, "seed-to-soil map")
    fertilizer = map(soil, "soil-to-fertilizer map")
    water = map(fertilizer, "fertilizer-to-water map")
    light = map(water, "water-to-light map")
    temperature = map(light, "light-to-temperature map")
    humidity = map(temperature, "temperature-to-humidity map")
    location = map(humidity, "humidity-to-location map")

    return soil, fertilizer, water, light, temperature, humidity, location

In [ ]:
locations = []
for seed in seeds:
    _, _, _, _, _, _, location = map_seed(seed)
    locations.append(location)

In [ ]:
min(locations)

## Part 2

In [856]:
from itertools import islice

def batched(iterable, n):
    it = iter(iterable)
    while batch := list(islice(it, n)):
        yield batch

In [916]:
TEST_INFILE = "inputs/day_5_test_1.txt"
INFILE = "inputs/day_5_1.txt"

#with open(TEST_INFILE) as infile:
with open(INFILE) as infile:
    lines = infile.read().split("\n\n")

In [917]:
maps = {line.split(":")[0]: line.split(":")[1].strip().split("\n") for line in lines}
seeds = [int(s) for s in maps.pop("seeds")[0].split()]

In [918]:
maps, seeds

({'seed-to-soil map': ['2797638787 1764015146 26675178',
   '3378130613 0 34101494',
   '388521182 772535426 141493697',
   '1148410839 174832460 323494173',
   '4259440029 4076987921 35527267',
   '1664835130 2610603190 214355271',
   '530014879 1680432282 83582864',
   '1471905012 758583825 13951601',
   '3909335908 4187323696 107643600',
   '3834527400 4173068203 14255493',
   '2824313965 136499996 38332464',
   '191321127 498326633 7266716',
   '1562436628 34101494 102398502',
   '3224943851 3320680296 153186762',
   '4058150677 3747050674 42920787',
   '2360258758 2578478063 32125127',
   '1485856613 682003810 76580015',
   '613597743 1875739628 457683244',
   '4016979508 3789971461 35854425',
   '3412232107 562825709 61634951',
   '4101071464 3825825886 158368565',
   '3848782893 4112515188 60553015',
   '3119382417 1629353314 51078968',
   '1879190401 1555705358 58037106',
   '2169686033 2387905338 190572725',
   '2900098047 914029123 217063885',
   '1071280987 2824958461 771298

In [919]:
class Range:
    def __init__(self, start, length):
        self.start = start
        self.length = length
        self.end = start + length - 1

    @classmethod
    def from_ends(cls, start, end):
        length = end - start + 1
        return cls(start, length)

    def includes(self, num):
        if num >= self.start and num < self.start + self.length:
            return True
        return False
    
    def __eq__(self, other):
        if not isinstance(other, Range):
            return NotImplemented
        return (self.start, self.length) == (other.start, other.length)

    def __repr__(self):
        return f"Range(start={self.start}, end={self.end}, length={self.length})"
    
    def __and__(self, other) -> tuple[tuple[Range]]:
        # we return a tuple;  first part is the part that should be mapped via
        # destination;  second is the part that maps to itself

        # self is completely inside of other so just return self to be mapped
        if other.start <= self.start and other.end >= self.end:
            return Range.from_ends(self.start, self.end), tuple()

        # self overlaps left edge of other
        if self.start < other.start and self.end >= other.start and self.end < other.end:
            return (
                Range.from_ends(other.start, self.end),
                (Range.from_ends(self.start, other.start - 1), )
            )
        
        # self overlaps right edge of other
        if self.start > other.start and self.end > other.end and self.start <= other.end:
            return (
                Range.from_ends(self.start, other.end),
                (Range.from_ends(other.end + 1, self.end), )
            )
        
        # self extends beyond both edges of other
        if self.start < other.start and self.end > other.end:
            return (
                (Range.from_ends(other.start, other.end)),
                (
                    Range.from_ends(self.start, other.start - 1),
                    Range.from_ends(other.end + 1, self.end)
                )
            )
        
        # we don't overlap at all
        return (
            tuple(),
            (Range.from_ends(self.start, self.end), )
        )


class SrcDestRanges:
    def __init__(self, src_range, dest_range):
        assert src_range.length == dest_range.length
        self.src  = src_range
        self.dest = dest_range

    def get_dest_from_src(self, src):
        if self.src.includes(src):
            return self.dest.start + (src - self.src.start)
        return None
    
    def get_dest_range_from_src(self, src: Range) -> Range:
        dest_start = self.get_dest_from_src(src.start)
        return Range(dest_start, src.length)
    
    def __repr__(self):
        return f"""
        Src  Range(start={self.src.start}, end={self.src.end}, length={self.src.length})
        Dest Range(start={self.dest.start}, end={self.dest.end}, length={self.dest.length})
        """

In [920]:
assert Range.from_ends(2, 4) & Range.from_ends(3, 5) == (
    Range.from_ends(3, 4),
    (Range.from_ends(2, 2), )
)

assert Range.from_ends(3, 5) & Range.from_ends(2, 4) == (
    Range.from_ends(3, 4),
    (Range.from_ends(5, 5), )
)

assert Range.from_ends(3, 4) & Range.from_ends(0, 10) == (
    Range.from_ends(3, 4),
    tuple()
)

assert Range.from_ends(0, 10) & Range.from_ends(3, 4) == (
    Range.from_ends(3, 4),
    (Range.from_ends(0, 2), Range.from_ends(5, 10)),
)

assert Range.from_ends(start=81, end=94) & Range.from_ends(start=15, end=51) == (
    tuple(),
    (Range.from_ends(start=81, end=94), )
)

In [921]:
for map_name, ranges in maps.items():
    range_objs = []
    for range in ranges:
        src  = Range(int(range.split()[1]), int(range.split()[2]))
        dest = Range(int(range.split()[0]), int(range.split()[2]))
        range = SrcDestRanges(src, dest)

        range_objs.append(range)
    maps[map_name] = range_objs

In [922]:
seed_ranges = []
for start, length in batched(seeds, 2):
    seed_ranges.append(Range(start, length))

In [923]:
seed_ranges

[Range(start=768975, end=37650595, length=36881621),
 Range(start=56868281, end=112255064, length=55386784),
 Range(start=1828225758, end=2912431314, length=1084205557),
 Range(start=2956956868, end=3084127619, length=127170752),
 Range(start=1117192172, end=1449752815, length=332560644),
 Range(start=357791695, end=487772340, length=129980646),
 Range(start=819363529, end=828508785, length=9145257),
 Range(start=993170544, end=1063815277, length=70644734),
 Range(start=3213715789, end=3525832661, length=312116873),
 Range(start=3107544690, end=3166904304, length=59359615)]

In [ ]:
def map(input_range, map_layer_name):
    this_map = maps[map_layer_name]

    unmapped = [input_range]
    mapped = []
    while len(unmapped) > 0:
        to_map = unmapped.pop()
        hit = False
        
        for this_map_row in this_map:
            intersected = to_map & this_map_row.src
            if intersected[0] != tuple():
                hit = True
                mapped.append(this_map_row.get_dest_range_from_src(intersected[0]))
                # if there's a leftover part we need to try to map it later
                if len(intersected[1]) > 0:
                    unmapped += [e for e in intersected[1]]
                # break out of the for--we don't need to check other maps
                break

        # if we checked all the maps and didn't get a hit we map it
        # directed to itself
        if not hit:
            mapped += [e for e in intersected[1]]

    return mapped


def map_seed(seed):
    soils = map(seed, "seed-to-soil map")

    fertilizers = []
    for s in soils:
        mapped = map(s, "soil-to-fertilizer map")
        fertilizers += mapped

    waters = []
    for f in fertilizers:
        mapped = map(f, "fertilizer-to-water map")
        waters += mapped

    lights = []
    for w in waters:
        mapped = map(w, "water-to-light map")
        lights += mapped

    temperatures = []
    for l in lights:
        mapped = map(l, "light-to-temperature map")
        temperatures += mapped

    humidities = []
    for t in temperatures:
        mapped = map(t, "temperature-to-humidity map")
        humidities += mapped

    locations = []
    for h in humidities:
        mapped = map(h, "humidity-to-location map")
        locations += mapped

    #return soils, fertilizers, waters, lights, temperatures, humidities, locations
    return min(l.start for l in locations)

In [931]:
mins = []
for s in seed_ranges:
    mins.append(map_seed(s))

In [932]:
mins

[550427451,
 125675349,
 31161857,
 110558445,
 1666246387,
 1502242005,
 1563561834,
 344110697,
 31161857,
 2704014460]

In [933]:
min(mins)

31161857